# Mood prediction with TensorFlow
Trying to predict the mood of a user each day with:
- Mood rated between 0 and 5
- Weather rated between 0 and 5
- Temperature
- Month of the year
- Day of the week between 0 and 6

In [1]:
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input

In [2]:
mood_history = pd.read_csv('mood-history.csv').sort_values('timestamp')
mood_history.head()

,timestamp,mood,weather,temperature,month,day,weekday
0,1616320024,5,5,8,2,21,6
1,1616406424,3,4,8,2,22,0
2,1616579224,4,5,9,2,23,1
3,1616579224,3,5,9,2,24,2
4,1616665624,2,4,7,2,25,3


## Data normalization

Normalizes some input features to make them all between 0 and 1.

Applies `(x - min) / (max - min)` for all the features.

In [3]:
def init_bag(size, index):
    """
    Initialize a bag of a specified size with an index in which the value
    will be 1.
    """

    bag = [0] * size
    bag[index] = 1

    return bag

def normalize_data(df):
    """
    Normalize the given Data Frame to put all values between 0 and 1.
    Creates bags for the months, days and week days.
    """

    if "mood" in df.columns:
        df['mood'] = df['mood'].apply(lambda x: x / 5)
    df['weather'] = df['weather'].apply(lambda x: x / 5)
    df['temperature'] = df['temperature'].apply(lambda x: (x + 30) / (50 + 30))

    df['month'] = df['month'].apply(lambda x: init_bag(12, x))
    df['day'] = df['day'].apply(lambda x: init_bag(31, x))
    df['weekday'] = df['weekday'].apply(lambda x: init_bag(7, x))

    return df

normalized_mood_history = normalize_data(mood_history)
normalized_mood_history

,timestamp,mood,weather,temperature,month,day,weekday
0,1616320024,1.0,1.0,0.4750,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 1]"
1,1616406424,0.6,0.8,0.4750,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0]"
2,1616579224,0.8,1.0,0.4875,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0]"
3,1616579224,0.6,1.0,0.4875,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0]"
4,1616665624,0.4,0.8,0.4625,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0]"
5,1616752024,0.4,0.6,0.4375,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0]"
6,1616838424,1.0,1.0,0.5250,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0]"


In [4]:
def get_training_data(df):
    """
    Flatten, removes columns names for integers to get the training data.
    """

    train = df[[
                'weather',
                'temperature',
                'month',
                'day',
                'weekday'
    ]]
    # Create new columns as bags for months, days and weekday
    train[[f'm{i}' for i in range(0, 12)]] = train['month'].apply(pd.Series)
    train[[f'd{i}' for i in range(0, 31)]] = train['day'].apply(pd.Series)
    train[[f'w{i}' for i in range(0, 7)]] = train['weekday'].apply(pd.Series)

    # Drop the old columns
    train.drop(['month', 'day', 'weekday'], inplace=True, axis=1)

    # Remove columns names for integers
    train.columns = range(train.shape[1])

    return train

X_train = get_training_data(normalized_mood_history)
X_train.head()

/opt/anaconda3/envs/datalore-user/lib/python3.7/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,1.0,0.4750,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0.8,0.4750,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1.0,0.4875,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1.0,0.4875,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0.8,0.4625,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [5]:
Y_train = mood_history[[
            'mood'
]]
Y_train.columns = range(Y_train.shape[1])

Y_train.head()

,0
0,1.0
1,0.6
2,0.8
3,0.6
4,0.4


In [6]:
model = Sequential()

model.add(Input(shape=(X_train.shape[1],)))
model.add(Dense(15, activation='sigmoid'))
model.add(Dense(1))

In [7]:
model.compile(optimizer='sgd', loss='mean_squared_error')

In [8]:
model.fit(X_train, Y_train, batch_size=1, epochs=10)

Epoch 1/10
7/7 [==============================] - 0s 1ms/step - loss: 0.4537
Epoch 2/10
7/7 [==============================] - 0s 1ms/step - loss: 0.1565
Epoch 3/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0962
Epoch 4/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0849
Epoch 5/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0806
Epoch 6/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0784
Epoch 7/10
7/7 [==============================] - 0s 2ms/step - loss: 0.0776
Epoch 8/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0758
Epoch 9/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0738
Epoch 10/10
7/7 [==============================] - 0s 1ms/step - loss: 0.0742


In [9]:
future_data = pd.DataFrame([[
            5,
            12,
            3,
            28,
            6       
]], columns=['weather', 'temperature', 'month', 'day', 'weekday'])
future_data = normalize_data(future_data)
future_data = get_training_data(future_data)

predictions = model.predict(future_data)

predictions

array([[0.6841199]], dtype=float32)